In [ ]:
from IPython.utils import io
with io.capture_output() as captured:
    !pip install scispacy
    !pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.2.4/en_core_sci_lg-0.2.4.tar.gz

In [ ]:
import numpy as np 
import pandas as pd

from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

import scispacy
import spacy
import en_core_sci_lg

from scipy.spatial.distance import jensenshannon

import joblib

from IPython.display import HTML, display

from ipywidgets import interact, Layout, HBox, VBox, Box
import ipywidgets as widgets
from IPython.display import clear_output

from tqdm import tqdm
from os.path import isfile

import seaborn as sb
import matplotlib.pyplot as plt
plt.style.use("dark_background")

In [ ]:
df = pd.read_csv('../input/cord-19-create-dataframe/cord19_df.csv')

In [ ]:
df.info()

In [ ]:
# medium model
nlp = en_core_sci_lg.load(disable=["tagger", "parser", "ner"])
nlp.max_length = 2000000

In [ ]:
def spacy_tokenizer(sentence):
    return [word.lemma_ for word in nlp(sentence) if not (word.like_num or word.is_stop or word.is_punct or word.is_space or len(word)==1)]

In [ ]:
# New stop words list 
customize_stop_words = [
    'doi', 'preprint', 'copyright', 'peer', 'reviewed', 'org', 'https', 'et', 'al', 'author', 'figure', 
    'rights', 'reserved', 'permission', 'used', 'using', 'biorxiv', 'medrxiv', 'license', 'fig', 'fig.', 'al.', 'Elsevier', 'PMC', 'CZI',
    '-PRON-'
]

# Mark them as stop words
for w in customize_stop_words:
    nlp.vocab[w].is_stop = True

In [ ]:
filepath = '../input/topic-modeling-finding-related-articles/'

In [ ]:
if not (isfile(filepath + 'vectorizer.csv') & isfile(filepath + 'data_vectorized.csv')):
    print('Files not there: generating')
    vectorizer = CountVectorizer(tokenizer = spacy_tokenizer, max_features=800000)
    data_vectorized = vectorizer.fit_transform(tqdm(all_texts))
    joblib.dump(vectorizer, 'vectorizer.csv')
    joblib.dump(data_vectorized, 'data_vectorized.csv')

else:
    vectorizer = joblib.load(filepath + 'vectorizer.csv')
    data_vectorized = joblib.load(filepath + 'data_vectorized.csv')

In [ ]:
# Train/Load Model
if not (isfile(filepath + 'lda.csv')):
    print('File not there: generating')
    lda = LatentDirichletAllocation(n_components=50, random_state=0)
    lda.fit(data_vectorized)

    joblib.dump(lda, 'lda.csv')

else:
    lda = joblib.load(filepath + 'lda.csv')

In [ ]:
def print_top_words(model, vectorizer, n_top_words):
    feature_names = vectorizer.get_feature_names()
    for topic_idx, topic in enumerate(model.components_):
        message = "\nTopic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()

In [ ]:
print_top_words(lda, vectorizer, n_top_words=25)

In [ ]:
if not (isfile(filepath + 'doc_topic_dist.csv')):
        print('File not there: generating')
        doc_topic_dist = pd.DataFrame(lda.transform(data_vectorized))
        doc_topic_dist.to_csv('doc_topic_dist.csv', index=False)
else:
        doc_topic_dist = pd.read_csv(filepath + 'doc_topic_dist.csv')  

In [ ]:
doc_topic_dist.head()

In [ ]:
is_covid19_article = df.body_text.str.contains('COVID-19|SARS-CoV-2|2019-nCov|SARS Coronavirus 2|2019 Novel Coronavirus')

In [ ]:
def get_k_nearest_docs(doc_dist, k=5, lower=1950, upper=2020, only_covid19=True, get_dist=True):
    '''
    doc_dist: topic distribution (sums to 1) of one article
    
    Returns the index of the k nearest articles (as by Jensen–Shannon divergence in topic space). 
    '''
    
    relevant_time = df.publish_year.between(lower, upper)
    
    if only_covid19:
        temp = doc_topic_dist[relevant_time & is_covid19_article]
    else:
        temp = doc_topic_dist[relevant_time]
         
    distances = temp.apply(lambda x: jensenshannon(x, doc_dist), axis=1)
    k_nearest = distances[distances != 0].nsmallest(n=k).index
    
    if get_dist:
        k_distances = distances[distances != 0].nsmallest(n=k)
        return k_nearest, k_distances
    else:
        return k_nearest

In [ ]:
# d = get_k_nearest_docs(doc_topic_dist[df.paper_id == '90b5ecf991032f3918ad43b252e17d1171b4ea63'].iloc[0])

# sb.kdeplot(d)

In [ ]:
def plot_article_dna(paper_id, width=20):
    t = df[df.paper_id == paper_id].title.values[0]
    doc_topic_dist[df.paper_id == paper_id].T.plot(kind='bar', legend=None, title=t, figsize=(width, 4))
    plt.xlabel('Topic')

def compare_dnas(paper_id, recommendation_id, width=20):
    t = df[df.paper_id == recommendation_id].title.values[0]
    temp = doc_topic_dist[df.paper_id == paper_id]
    ymax = temp.max(axis=1).values[0]*1.25
    temp = pd.concat([temp, doc_topic_dist[df.paper_id == recommendation_id]])
    temp.T.plot(kind='bar', title=t, figsize=(width, 4), ylim= [0, ymax])
    plt.xlabel('Topic')
    plt.legend(['Selection', 'Recommendation'])

# compare_dnas('90b5ecf991032f3918ad43b252e17d1171b4ea63', 'a137eb51461b4a4ed3980aa5b9cb2f2c1cf0292a')

def dna_tabs(paper_ids):
    k = len(paper_ids)
    outs = [widgets.Output() for i in range(k)]

    tab = widgets.Tab(children = outs)
    tab_titles = ['Paper ' + str(i+1) for i in range(k)]
    for i, t in enumerate(tab_titles):
        tab.set_title(i, t)
    display(tab)

    for i, t in enumerate(tab_titles):
        with outs[i]:
            ax = plot_article_dna(paper_ids[i])
            plt.show(ax)

def compare_tabs(paper_id, recommendation_ids):
    k = len(recommendation_ids)
    outs = [widgets.Output() for i in range(k)]

    tab = widgets.Tab(children = outs)
    tab_titles = ['Paper ' + str(i+1) for i in range(k)]
    for i, t in enumerate(tab_titles):
        tab.set_title(i, t)
    display(tab)

    for i, t in enumerate(tab_titles):
        with outs[i]:
            ax = compare_dnas(paper_id, recommendation_ids[i])
            plt.show(ax)

In [ ]:
def recommendation(paper_id, k=5, lower=1950, upper=2020, only_covid19=True, plot_dna=False):
    '''
    Returns the title of the k papers that are closest (topic-wise) to the paper given by paper_id.
    '''
    
    print(df.title[df.paper_id == paper_id].values[0])

    recommended, dist = get_k_nearest_docs(doc_topic_dist[df.paper_id == paper_id].iloc[0], k, lower, upper, only_covid19, get_dist=True)
    recommended = df.iloc[recommended].copy()
    recommended['similarity'] = 1 - dist 
    
    h = '<br/>'.join(['<a href="' + l + '" target="_blank">'+ n + '</a>' +' (Similarity: ' + "{:.2f}".format(s) + ')' for l, n, s in recommended[['url','title', 'similarity']].values])
    display(HTML(h))
    
    if plot_dna:
        compare_tabs(paper_id, recommended.paper_id.values)

In [ ]:
def related_papers():
    '''
    Creates a widget where you can select one of many papers about covid-19 and then displays related articles from the whole dataset.
    '''
    covid_papers = df[df.body_text.str.contains('COVID-19|SARS-CoV-2|2019-nCov|SARS Coronavirus 2|2019 Novel Coronavirus')][['paper_id', 'title']] # are there more names?
    title_to_id = covid_papers.set_index('title')['paper_id'].to_dict()
    
    def main_function(bullet, k=5, year_range=[1950, 2020], only_covid19=True):
        recommendation(title_to_id[bullet], k, lower=year_range[0], upper=year_range[1], only_covid19=only_covid19)
    
    yearW = widgets.IntRangeSlider(min=1950, max=2020, value=[2010, 2020], description='Year Range', 
                                   continuous_update=False, layout=Layout(width='40%'))
    covidW = widgets.Checkbox(value=False,description='Only COVID-19-Papers',disabled=False, indent=False, layout=Layout(width='20%'))
    kWidget = widgets.IntSlider(value=10, description='k', max=50, min=1, layout=Layout(width='20%'))

    bulletW = widgets.Select(options=title_to_id.keys(), layout=Layout(width='90%', height='200px'), description='Title:')

    widget = widgets.interactive(main_function, bullet=bulletW, k=kWidget, year_range=yearW, only_covid19=covidW)

    controls = VBox([Box(children=[widget.children[:-1][1], widget.children[:-1][2], widget.children[:-1][3]], 
                         layout=Layout(justify_content='space-around')), widget.children[:-1][0]])
    output = widget.children[-1]
    display(VBox([controls, output]))

In [ ]:
task1 = ["Range of incubation periods for the disease in humans (and how this varies across age and health status) and how long individuals are contagious, even after recovery.",
"Prevalence of asymptomatic shedding and transmission (e.g., particularly children).",
"Seasonality of transmission.",
"Physical science of the coronavirus (e.g., charge distribution, adhesion to hydrophilic/phobic surfaces, environmental survival to inform decontamination efforts for affected areas and provide information about viral shedding).",
"Persistence and stability on a multitude of substrates and sources (e.g., nasal discharge, sputum, urine, fecal matter, blood).",
"Persistence of virus on surfaces of different materials (e,g., copper, stainless steel, plastic).",
"Natural history of the virus and shedding of it from an infected person",
"Implementation of diagnostics and products to improve clinical processes",
"Disease models, including animal models for infection, disease and transmission",
"Tools and studies to monitor phenotypic change and potential adaptation of the virus",
"Immune response and immunity",
"Effectiveness of movement control strategies to prevent secondary transmission in health care and community settings",
 "Effectiveness of personal protective equipment (PPE) and its usefulness to reduce risk of transmission in health care and community settings",
"Role of the environment in transmission"]

task2 = ['Data on potential risks factors',
'Smoking, pre-existing pulmonary disease',
'Co-infections (determine whether co-existing respiratory/viral infections make the virus more transmissible or virulent) and other co-morbidities',
'Neonates and pregnant women',
'Socio-economic and behavioral factors to understand the economic impact of the virus and whether there were differences.',
'Transmission dynamics of the virus, including the basic reproductive number, incubation period, serial interval, modes of transmission and environmental factors', 
'Severity of disease, including risk of fatality among symptomatic hospitalized patients, and high-risk patient groups',
'Susceptibility of populations',
'Public health mitigation measures that could be effective for control']

task3 = ['Real-time tracking of whole genomes and a mechanism for coordinating the rapid dissemination of that information to inform the development of diagnostics and therapeutics and to track variations of the virus over time.',
'Access to geographic and temporal diverse sample sets to understand geographic distribution and genomic differences, and determine whether there is more than one strain in circulation. Multi-lateral agreements such as the Nagoya Protocol could be leveraged.',
'Evidence that livestock could be infected (e.g., field surveillance, genetic sequencing, receptor binding) and serve as a reservoir after the epidemic appears to be over.',
'Evidence of whether farmers are infected, and whether farmers could have played a role in the origin.',
'Surveillance of mixed wildlife- livestock farms for SARS-CoV-2 and other coronaviruses in Southeast Asia.',
'Experimental infections to test host range for this pathogen.',
'Animal host(s) and any evidence of continued spill-over to humans',
'Socioeconomic and behavioral risk factors for this spill-over',
'Sustainable risk reduction strategies']

task4 = ["Guidance on ways to scale up NPIs in a more coordinated way (e.g., establish funding, infrastructure and authorities to support real time, authoritative (qualified participants) collaboration with all states to gain consensus on consistent guidance and to mobilize resources to geographic areas where critical shortfalls are identified) to give us time to enhance our health care delivery system capacity to respond to an increase in cases.",
"Rapid design and execution of experiments to examine and compare NPIs currently being implemented. DHS Centers for Excellence could potentially be leveraged to conduct these experiments.",
"Rapid assessment of the likely efficacy of school closures, travel bans, bans on mass gatherings of various sizes, and other social distancing approaches.",
"Methods to control the spread in communities, barriers to compliance and how these vary among different populations..",
"Models of potential interventions to predict costs and benefits that take account of such factors as race, income, disability, age, geographic location, immigration status, housing status, employment status, and health insurance status.",
"Policy changes necessary to enable the compliance of individuals with limited resources and the underserved with NPIs.",
"Research on why people fail to comply with public health advice, even if they want to do so (e.g., social or financial costs may be too high).",
"Research on the economic impact of this or any pandemic. This would include identifying policy and programmatic alternatives that lessen/mitigate risks to critical government services, food distribution and supplies, access to critical household supplies, and access to health diagnoses, treatment, and needed care, regardless of ability to pay."]

task5 = ["Effectiveness of drugs being developed and tried to treat COVID-19 patients. Clinical and bench trials to investigate less common viral inhibitors against COVID-19 such as naproxen, clarithromycin, and minocyclinethat that may exert effects on viral replication.",
"Methods evaluating potential complication of Antibody-Dependent Enhancement (ADE) in vaccine recipients.",
"Exploration of use of best animal models and their predictive value for a human vaccine.",
"Capabilities to discover a therapeutic (not vaccine) for the disease, and clinical effectiveness studies to discover therapeutics, to include antiviral agents.",
"Alternative models to aid decision makers in determining how to prioritize and distribute scarce, newly proven therapeutics as production ramps up. This could include identifying approaches for expanding production capacity to ensure equitable and timely distribution to populations in need.",
"Efforts targeted at a universal coronavirus vaccine.",
"Efforts to develop animal models and standardize challenge studies",
"Efforts to develop prophylaxis clinical studies and prioritize in healthcare workers",
"Approaches to evaluate risk for enhanced disease after vaccination",
"Assays to evaluate vaccine immune response and process development for vaccines, alongside suitable animal models [in conjunction with therapeutics]"]

task6 = ["Efforts to articulate and translate existing ethical principles and standards to salient issues in COVID-2019", 
"Efforts to embed ethics across all thematic areas, engage with novel ethical issues that arise and coordinate to minimize duplication of oversight",
"Efforts to support sustained education, access, and capacity building in the area of ethics",
"Efforts to establish a team at WHO that will be integrated within multidisciplinary research and operational platforms and that will connect with existing and expanded global networks of social sciences.",
"Efforts to develop qualitative assessment frameworks to systematically collect information related to local barriers and enablers for the uptake and adherence to public health measures for prevention and control. This includes the rapid identification of the secondary impacts of these measures. (e.g. use of surgical masks, modification of health seeking behaviors for SRH, school closures)",
"Efforts to identify how the burden of responding to the outbreak and implementing public health measures affects the physical and psychological health of those providing care for Covid-19 patients and identify the immediate needs that must be addressed.",
"Efforts to identify the underlying drivers of fear, anxiety and stigma that fuel misinformation and rumor, particularly through social media."]

task7 = ["How widespread current exposure is to be able to make immediate policy recommendations on mitigation measures. Denominators for testing and a mechanism for rapidly sharing that information, including demographics, to the extent possible. Sampling methods to determine asymptomatic disease (e.g., use of serosurveys (such as convalescent samples) and early detection of disease (e.g., use of screening of neutralizing antibodies such as ELISAs).",
"Efforts to increase capacity on existing diagnostic platforms and tap into existing surveillance platforms.",
"Recruitment, support, and coordination of local expertise and capacity (public, private—commercial, and non-profit, including academic), including legal, ethical, communications, and operational issues.",
"National guidance and guidelines about best practices to states (e.g., how states might leverage universities and private laboratories for testing purposes, communications to public health officials and the public).",
"Development of a point-of-care test (like a rapid influenza test) and rapid bed-side tests, recognizing the tradeoffs between speed, accessibility, and accuracy.",
"Rapid design and execution of targeted surveillance experiments calling for all potential testers using PCR in a defined area to start testing and report to a specific entity. These experiments could aid in collecting longitudinal samples, which are critical to understanding the impact of ad hoc local interventions (which also need to be recorded).",
"Separation of assay development issues from instruments, and the role of the private sector to help quickly migrate assays onto those devices.",
"Efforts to track the evolution of the virus (i.e., genetic drift or mutations) and avoid locking into specific reagents and surveillance/detection schemes.",
"Latency issues and when there is sufficient viral load to detect the pathogen, and understanding of what is needed in terms of biological and environmental sampling.",
"Use of diagnostics such as host response markers (e.g., cytokines) to detect early disease or predict severe disease progression, which would be important to understanding best clinical practice and efficacy of therapeutic interventions.",
"Policies and protocols for screening and testing.",
"Policies to mitigate the effects on supplies associated with mass testing, including swabs and reagents.",
"Technology roadmap for diagnostics.",
"Barriers to developing and scaling up new diagnostic tests (e.g., market forces), how future coalition and accelerator models (e.g., Coalition for Epidemic Preparedness Innovations) could provide critical funding for diagnostics, and opportunities for a streamlined regulatory environment.",
"New platforms and technology (e.g., CRISPR) to improve response times and employ more holistic approaches to COVID-19 and future diseases.",
"Coupling genomics and diagnostic testing on a large scale.",
"Enhance capabilities for rapid sequencing and bioinformatics to target regions of the genome that will allow specificity for a particular variant.",
"Enhance capacity (people, technology, data) for sequencing with advanced analytics for unknown pathogens, and explore capabilities for distinguishing naturally-occurring pathogens from intentional.",
"One Health surveillance of humans and potential sources of future spillover or ongoing exposure for this organism and future pathogens, including both evolutionary hosts (e.g., bats) and transmission hosts (e.g., heavily trafficked and farmed wildlife and domestic food and companion species), inclusive of environmental, demographic, and occupational risk factors."]

task8 = ["Resources to support skilled nursing facilities and long term care facilities.",
"Mobilization of surge medical staff to address shortages in overwhelmed communities",
"Age-adjusted mortality data for Acute Respiratory Distress Syndrome (ARDS) with/without other organ failure – particularly for viral etiologies",
"Extracorporeal membrane oxygenation (ECMO) outcomes data of COVID-19 patients",
"Outcomes data for COVID-19 after mechanical ventilation adjusted for age.",
"Knowledge of the frequency, manifestations, and course of extrapulmonary manifestations of COVID-19, including, but not limited to, possible cardiomyopathy and cardiac arrest.",
"Application of regulatory standards (e.g., EUA, CLIA) and ability to adapt care to crisis standards of care level.",
"Approaches for encouraging and facilitating the production of elastomeric respirators, which can save thousands of N95 masks.",
"Best telemedicine practices, barriers and faciitators, and specific actions to remove/expand them within and across state boundaries.",
"Guidance on the simple things people can do at home to take care of sick people and manage disease.",
"Oral medications that might potentially work.",
"Use of AI in real-time health care delivery to evaluate interventions, risk factors, and outcomes in a way that could not be done manually.",
"Best practices and critical challenges and innovative solutions and technologies in hospital flow and organization, workforce protection, workforce allocation, community-based support resources, payment, and supply chain management to enhance capacity, efficiency, and outcomes.",
"Efforts to define the natural history of disease to inform clinical care, public health interventions, infection prevention control, transmission, and clinical trials",
"Efforts to develop a core clinical outcome set to maximize usability of data across a range of trials",
"Efforts to determine adjunctive and supportive interventions that can improve the clinical outcomes of infected patients (e.g. steroids, high flow oxygen)"]

task9 = ["Methods for coordinating data-gathering with standardized nomenclature.",
"Sharing response information among planners, providers, and others.",
"Understanding and mitigating barriers to information-sharing.",
"How to recruit, support, and coordinate local (non-Federal) expertise and capacity relevant to public health emergency response (public, private, commercial and non-profit, including academic).",
"Integration of federal/state/local public health surveillance systems.",
"Value of investments in baseline public health response infrastructure preparedness",
"Modes of communicating with target high-risk populations (elderly, health care workers).",
"Risk communication and guidelines that are easy to understand and follow (include targeting at risk populations’ families too).",
"Communication that indicates potential risk of disease to all population groups.",
"Misunderstanding around containment and mitigation.",
"Action plan to mitigate gaps and problems of inequity in the Nation’s public health capability, capacity, and funding to ensure all citizens in need are supported and can access information, surveillance, and treatment.",
"Measures to reach marginalized and disadvantaged populations.",
"Data systems and research priorities and agendas incorporate attention to the needs and circumstances of disadvantaged populations and underrepresented minorities.",
"Mitigating threats to incarcerated people from COVID-19, assuring access to information, prevention, diagnosis, and treatment.",
"Understanding coverage policies (barriers and opportunities) related to testing, treatment, and care"]

tasks={'What is known about transmission, incubation, and environmental stability?': task1,
       'What do we know about COVID-19 risk factors?': task2, 
       'What do we know about virus genetics, origin, and evolution?': task3, 
       'What do we know about non-pharmaceutical interventions?': task4,
       'What do we know about vaccines and therapeutics?': task5, 
       'What has been published about ethical and social science considerations?': task6, 
       'What do we know about diagnostics and surveillance?': task7,
       'What has been published about medical care?': task8, 
       'What has been published about information sharing and inter-sectoral collaboration?': task9}

In [ ]:
def relevant_articles(tasks, k=3, lower=1950, upper=2020, only_covid19=True):
    tasks = [tasks] if type(tasks) is str else tasks 
    
    tasks_vectorized = vectorizer.transform(tasks)
    tasks_topic_dist = pd.DataFrame(lda.transform(tasks_vectorized))

    for index, bullet in enumerate(tasks):
        print(bullet)
        recommended, distances = get_k_nearest_docs(tasks_topic_dist.iloc[index], k, lower, upper, only_covid19, get_dist=True)
        recommended = df.iloc[recommended]

        h = '<br/>'.join(['<a href="' + l + '" target="_blank">'+ n + '</a>' for l, n in recommended[['url','title']].values])
        display(HTML(h))
        print(distances)

In [ ]:
relevant_articles("What is the influence of Smoking, pre-existing pulmonary disease on Covid-19", 1, only_covid19=True)

In [ ]:
relevant_articles("Can cold weather and snow prevent the coronavirus disease?", 5, only_covid19=True)

In [ ]:
def get_context(task, k=3, lower=1950, upper=2020, only_covid19=True):
    if type(task) == str:
        task = [task]
        task_vectorized = vectorizer.transform(task)
        task_topic_dist = pd.DataFrame(lda.transform(task_vectorized))
        recommended, _ = get_k_nearest_docs(task_topic_dist.iloc[0], k, lower, upper, only_covid19)
        recommended = df.iloc[recommended]
        return recommended['title'].values[0], recommended['body_text'].values[0].replace("\n", " ")
    else :
        raise Exception("please enter a task as a string.")

In [ ]:
get_context("What is the influence of Smoking, pre-existing pulmonary disease on Covid-19", 1, only_covid19=True)

In [ ]:
question_list = """ What are the potential risks factors of Covid-19
What is the influence of Smoking, pre-existing pulmonary disease on Covid-19
Will Co-infections and other co-morbidities co-existing respiratory/viral infections make the virus more transmissible or virulent 
Risk factors associated with Neonates and pregnant women
What are the risk factors associated with Socio-economic and behavioral factors to understand the economic impact of Covid19.
What are the Transmission dynamics of the virus, including the basic reproductive number, incubation period, serial interval, modes of transmission and environmental factors
What is the Severity of disease, including risk of fatality among symptomatic hospitalized patients, and high-risk patient groups
What is Susceptibility of populations for covid19 rik factors
What are Public health mitigation measures that could be effective for control of covid19"""

question_list = question_list.split("\n")
qa_dataframe = pd.DataFrame({"Questions": question_list})

In [ ]:
qa_dataframe["Context"] = qa_dataframe["Questions"].apply(lambda x : get_context(x, 1, only_covid19=True)[1])

In [ ]:
qa_dataframe["title"] = qa_dataframe["Questions"].apply(lambda x : get_context(x, 1, only_covid19=True)[0])

In [ ]:
qa_dataframe["title"][1]

 # BIO BERT CODE STARTS HERE

In [ ]:
for dirname, _, filenames in os.walk('/kaggle/input/bertmodel/factoid_output'):
    print(dirname, filenames)

In [ ]:
"""Question: What do we know about COVID-19 risk factors?
Data on potential risks factors
Smoking, pre-existing pulmonary disease
Co-infections (determine whether co-existing respiratory/viral infections make the virus more transmissible or virulent) and other co-morbidities 
Neonates and pregnant women
Socio-economic and behavioral factors to understand the economic impact of the virus and whether there were differences.
Transmission dynamics of the virus, including the basic reproductive number, incubation period, serial interval, modes of transmission and environmental factors

Severity of disease, including risk of fatality among symptomatic hospitalized patients, and high-risk patient groups
Susceptibility of populations
Public health mitigation measures that could be effective for control"""

In [ ]:

question_list = """ What are the potential risks factors of Covid-19
What is the influence of Smoking, pre-existing pulmonary disease on Covid-19
Will Co-infections and other co-morbidities co-existing respiratory/viral infections make the virus more transmissible or virulent 
Risk factors associated with Neonates and pregnant women
What are the risk factors associated with Socio-economic and behavioral factors to understand the economic impact of Covid19.
What are the Transmission dynamics of the virus, including the basic reproductive number, incubation period, serial interval, modes of transmission and environmental factors
What is the Severity of disease, including risk of fatality among symptomatic hospitalized patients, and high-risk patient groups
What is Susceptibility of populations for covid19 rik factors
What are Public health mitigation measures that could be effective for control of covid19"""

question_list = question_list.split("\n")
qa_dataframe = pd.DataFrame({"Questions": question_list})

In [ ]:
qa_dataframe["Context"] = qa_dataframe["Questions"].apply(lambda x : get_context(x, "title", model, papers_df)[0])

In [ ]:
qa_dataframe

In [ ]:
ls -l '/kaggle/input/predictionquestions-hc/CovidQuestions_HC_2.csv'

## **BIOBERT Fine-tuning and Prediction on SQUAD 2.0 using Cloud TPU!**

---

In [ ]:
#import tarfile

In [ ]:
#ls -l /kaggle/input/bertmodel/factoid_output

In [ ]:
import datetime
import json
import os
import pprint
import random
import string
import sys

In [ ]:
!pip install tensorflow==1.15.2

In [ ]:
import tensorflow as tf
print(tf.version)

In [ ]:
# detect and init the TPU
#tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
#tf.config.experimental_connect_to_cluster(tpu)
#tf.tpu.experimental.initialize_tpu_system(tpu)

Data preparation

In [ ]:
# covidQuestions = pd.read_csv('/kaggle/input/predictionquestions-hc-1/CovidQuestions_HC_2_1row.csv', encoding='ISO-8859-1')
# covidQuestions.head()

covidQuestions = qa_dataframe

In [ ]:
#covidQuestions["extracted_context"]

In [ ]:
! mkdir temp

In [ ]:
! ls -l temp

In [ ]:
covidQuestions.to_json('temp/QnAInput.json')

In [ ]:
#Test File

import json
from uuid import uuid4

def restructure_json_Test(input_data):   
    output = dict()
    data = []
    paragraphs = []
    for i in input_data["Questions"].keys():
        qas = []
        answers = []
      
        qas.append({"id" : str(uuid4()),
                   "question" : input_data["Questions"][i],
                   })
        paragraphs = paragraphs + [{"qas" : qas, "context" : input_data["Context"][i]}]
    data = data + [{"paragraphs" : paragraphs, "title" : "BioASQ6b"}]
    output["data"] = data
    output["version"] = "BioASQ6b"
    return output

In [ ]:
with open('temp/QnAInput.json',encoding='ISO-8859-1' ) as f:
    input_data = json.load(f)
type(input_data)

In [ ]:
outputData = restructure_json_Test(input_data)
with open('temp/QnAInputTest.json', 'w',  encoding='ISO-8859-1') as json_file:
    json.dump(outputData, json_file)

In [ ]:
BIOBERT_DIR = "/kaggle/input/bertmodel/factoid_output"
#BIOASQ_DIR = "/kaggle/working/task1/"
BIOASQ_INPUT_DIR = "/kaggle/input/CORD-19-research-challenge/"
BIOASQ_OUTPUT = "/kaggle/working/task1/BIOBERTQnA/"
BIOBERT_TRAINED_DIR = "/kaggle/input/bertmodel/factoid_output"

Inference for Que and Answer

In [ ]:
!ls -ltr /kaggle/input/biobertcode/GitRepo/BIOBERT/bioasq-biobert

In [ ]:
! ls -l '/kaggle/input/biobertcode/GitRepo/BIOBERT/bioasq-biobert/run_factoid.py'
! ls -l '/kaggle/input/biobertconfig/biobertconfig/vocab.txt'
! ls -l '/kaggle/input/biobertconfig/biobertconfig/bert_config.json'
! ls -l '/kaggle/input/bertmodel/factoid_output/model.ckpt-0.index'
#! ls -l '/kaggle/input/predictionquestions-hc/QnAInputTest.json'
! ls -l 'temp/QnAInputTest.json'

In [ ]:
biobert_code = "/kaggle/input/biobertcode/GitRepo/BIOBERT/bioasq-biobert"
biobert_config = "/kaggle/input/biobertconfig/biobertconfig/"

In [ ]:
! python /kaggle/input/biobertcode/GitRepo/BIOBERT/bioasq-biobert/run_factoid.py \
     --do_train=False \
     --do_predict=True \
     --vocab_file=/kaggle/input/biobertconfig/biobertconfig/vocab.txt \
     --bert_config_file=/kaggle/input/biobertconfig/biobertconfig/bert_config.json \
     --init_checkpoint=/kaggle/input/bertmodel/factoid_output/model.ckpt-0.index \
     --max_seq_length=384 \
     --train_batch_size=12 \
     --learning_rate=5e-6 \
     --doc_stride=128 \
     --num_train_epochs=1.0 \
     --do_lower_case=False \
     --train_file=$BIOASQ_INPUT_DIR/BioASQ-6b/train/Full-Abstract/BioASQ-train-factoid-6b-full-annotated.json \
     --predict_file=temp/QnAInputTest.json \
     --output_dir=/kaggle/output/kaggle/factoid_output/prediction/ \
     --max_answer_length=100

In [ ]:
ls -l /kaggle/output/kaggle/factoid_output/prediction/predictions.json

In [ ]:
#json.open("/kaggle/output/kaggle/factoid_output/prediction/predictions.json").read()
with open('/kaggle/output/kaggle/factoid_output/prediction/predictions.json',encoding='ISO-8859-1' ) as f:
    output_data = json.load(f)
type(output_data)

In [ ]:
output_data

In [ ]:
qa_dataframe

In [ ]:

with open('temp/QnAInputTest.json',encoding='ISO-8859-1' ) as f:
    output_data_questions = json.load(f)
test_data

In [ ]:
paragraphs = output_data_questions['data'][0]['paragraphs']

for i in paragraphs:
    id = i['qas'][0]['id']
    question = i['qas'][0]['question']
    answer = output_data[id]
    print("Question : ",question ,"\n Answer : ", answer)

In [ ]:
paragraphs[0]